In [1]:
# Load necessary libraries
# Libraries for data manipulation, visualization, and date-time handling
library(tidyverse)
library(lubridate)
library(ggplot2)
library(dplyr)
library(tidyr)

# Read and preprocess data
# Load the dataset and clean column names by replacing spaces with underscores
# Convert Date_Time to datetime format and ensure numeric columns are properly formatted
df <- read_csv("Air Quality Control/air_quality_data.csv") %>%
  rename_with(~ gsub("\\s+", "_", .)) %>% # Replace spaces with underscores in column names
  mutate(
    Date_Time = as_datetime(Date_Time),
    across(c(PM2.5, PM10, NO2, SO2, CO, AQI, Temperature, Humidity, Windspeed), as.numeric)
  )

# Handle missing values (simple imputation with means)
# Replace missing values in numeric columns with their respective column means
df <- df %>%
  mutate(across(where(is.numeric), ~ifelse(is.na(.), mean(., na.rm = TRUE), .)))

# Remove duplicates
# Ensure there are no duplicate rows in the dataset
df <- distinct(df)

# Deeper Analysis for Different Areas
# 1. Calculate maximum, minimum, and average AQI for each city
# Summarize AQI statistics for each city
city_aqi_stats <- df %>%
  group_by(City) %>%
  summarise(
    Max_AQI = max(AQI),
    Min_AQI = min(AQI),
    Avg_AQI = mean(AQI)
  )
print(city_aqi_stats)

# 2. Analyze pollutant trends over time for each city
# Reshape data to long format for pollutant levels and plot trends over time
pollutant_trends <- df %>%
  pivot_longer(cols = c(PM2.5, PM10, NO2, SO2, CO), names_to = "Pollutant", values_to = "Level")

ggplot(pollutant_trends, aes(x = Date_Time, y = Level, color = Pollutant)) +
  geom_line() +
  facet_wrap(~ City) +
  labs(title = "Pollutant Trends Over Time by City", x = "Date and Time", y = "Pollutant Level") +
  theme_minimal()

# Explanation: This visual shows how pollutant levels (PM2.5, PM10, NO2, SO2, CO) vary over time for each city. It helps identify patterns or spikes in pollution.

# 3. Distribution of weather parameters (Temperature, Humidity, Windspeed) for each city
# Reshape data to long format for weather parameters and plot density distributions
weather_distributions <- df %>%
  pivot_longer(cols = c(Temperature, Humidity, Windspeed), names_to = "Parameter", values_to = "Value")

ggplot(weather_distributions, aes(x = Value, fill = City)) +
  geom_density(alpha = 0.5) +
  facet_wrap(~ Parameter, scales = "free") +
  labs(title = "Weather Parameter Distributions by City", x = "Value", y = "Density") +
  theme_minimal()

# Explanation: This visual shows the distribution of weather parameters (Temperature, Humidity, Windspeed) for each city. It highlights variations in weather conditions across cities.

# 4. Correlation analysis between AQI and pollutants for each city
# Calculate correlations between AQI and pollutants for each city
city_correlations <- df %>%
  group_by(City) %>%
  summarise(
    Correlation_PM2.5 = cor(AQI, PM2.5),
    Correlation_PM10 = cor(AQI, PM10),
    Correlation_NO2 = cor(AQI, NO2),
    Correlation_SO2 = cor(AQI, SO2),
    Correlation_CO = cor(AQI, CO)
  )
print(city_correlations)

# Explanation: This summary table shows the strength of the relationship between AQI and individual pollutants for each city. High correlations indicate strong associations.

# 5. Visualize AQI distribution for each city
# Plot histogram of AQI values for each city
ggplot(df, aes(x = AQI, fill = City)) +
  geom_histogram(binwidth = 5, alpha = 0.7, position = "dodge") +
  labs(title = "AQI Distribution by City", x = "AQI", y = "Frequency") +
  theme_minimal()

# Explanation: This histogram shows the distribution of AQI values for each city. It helps identify cities with consistently high or low air quality levels.

# Additional EDA: Average pollution levels by city
# Calculate average levels of pollutants and AQI for each city
avg_pollution <- df %>%
  group_by(City) %>%
  summarise(
    avg_PM2.5 = mean(PM2.5, na.rm = TRUE),
    avg_PM10 = mean(PM10, na.rm = TRUE),
    avg_NO2 = mean(NO2, na.rm = TRUE),
    avg_SO2 = mean(SO2, na.rm = TRUE),
    avg_CO = mean(CO, na.rm = TRUE),
    avg_AQI = mean(AQI, na.rm = TRUE)
  ) %>%
  arrange(desc(avg_AQI))

# Explanation: This table provides a summary of average pollutant levels and AQI for each city, helping to rank cities by air quality.

# Visualize average pollutant levels by city
pollutant_comparison <- avg_pollution %>%
  pivot_longer(cols = starts_with("avg_"), names_to = "Pollutant", values_to = "Level")

ggplot(pollutant_comparison, aes(x = City, y = Level, fill = Pollutant)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Average Pollutant Levels by City", x = "City", y = "Average Level") +
  theme_minimal()

# Explanation: This bar chart compares average pollutant levels across cities, highlighting cities with higher pollution levels.

# Identify most polluted cities
most_polluted <- avg_pollution %>% slice_max(avg_AQI, n = 1)

# Correlation analysis
cor_matrix <- df %>%
  select(PM2.5, PM10, NO2, SO2, CO, AQI, Temperature, Humidity, Windspeed) %>%
  cor(use = "complete.obs")

# Temperature impact analysis
temp_impact <- df %>%
  mutate(temp_bin = cut(Temperature, breaks = 5)) %>%
  group_by(temp_bin) %>%
  summarise(avg_AQI = mean(AQI, na.rm = TRUE))

# Summary statistics
pollution_stats <- df %>%
  select(PM2.5, PM10, NO2, SO2, CO, AQI) %>%
  summary()

ggplot(df, aes(x = Date_Time, y = AQI, color = City)) +
  geom_line() +
  labs(title = "AQI Trends Over Time by City",
       x = "Date Time",
       y = "AQI Level") +
  theme_minimal()

df %>%
  pivot_longer(cols = c(PM2.5, PM10, NO2), 
               names_to = "Pollutant", 
               values_to = "Value") %>%
  ggplot(aes(x = City, y = Value, fill = Pollutant)) +
  geom_bar(stat = "summary", fun = "mean", position = "dodge") +
  labs(title = "Average Pollutant Levels by City",
       y = "Concentration (µg/m³)") +
  theme_minimal()

df %>%
  select(PM2.5, Temperature, Humidity, Windspeed) %>%
  GGally::ggpairs() +
  labs(title = "Relationships Between Pollution and Weather Factors") +
  theme_minimal()

print(colnames(df))

# Find worst pollution days
worst_days <- df %>%
  group_by(City) %>%
  select(City, Date_Time, AQI, PM2.5, PM10)

print(colnames(worst_days))
print(colnames(cor_matrix))

# Find correlations between weather and pollution
weather_cor <- cor_matrix %>%
  as.data.frame() %>%
  select(PM2.5, PM10, AQI) %>%
  rownames_to_column("Weather_Factor") %>%
  filter(Weather_Factor %in% c("Temperature", "Humidity", "Windspeed"))

# Create a summary report
report_summary <- list(
  "Most_Polluted_Cities" = avg_pollution,
  "Worst_Pollution_Days" = worst_days,
  "Weather_Impacts" = weather_cor,
  "Summary_Statistics" = pollution_stats
)

# Save visualizations
ggsave("aqi_trends.png", width = 10, height = 6)
ggsave("pollutant_comparison.png", width = 10, height = 6)
ggsave("weather_relationships.png", width = 8, height = 8)

cat(sprintf(
  "Our analysis reveals that %s has the worst air quality with an average AQI of %.1f, driven primarily by high levels of particulate matter (PM2.5 avg: %.1f µg/m³, PM10 avg: %.1f µg/m³).",
  most_polluted$City,
  most_polluted$avg_AQI,
  most_polluted$avg_PM2.5,
  most_polluted$avg_PM10
))

print(length(df$AQI))

library(forecast)

# Ensure there is enough data for decomposition
if (length(df$AQI) >= 14) {  # At least two weeks of data for weekly seasonality
  ts_data <- ts(df$AQI, frequency = 7)  # Weekly seasonality for daily data
  decomposed <- stl(ts_data, s.window = "periodic")
  autoplot(decomposed)
} else {
  cat("Not enough data for STL decomposition with weekly seasonality.\n")
}

# library(forecast)
# ts_data <- ts(df$AQI, frequency = 7)  # Weekly seasonality for daily data
# decomposed <- stl(ts_data, s.window = "periodic")
# autoplot(decomposed)

library(leaflet)
library(sf)
# Assuming we have coordinates for cities
leaflet(df) %>% 
  addTiles() %>%
  addHeatmap(lng = ~lon, lat = ~lat, intensity = ~PM2.5, radius = 15)

library(ppcor)
pcor.test(df$PM2.5, df$Windspeed, df$Temperature)

library(randomForest)
model <- randomForest(AQI ~ Temperature + Humidity + Windspeed, data = df)
varImpPlot(model)

df %>%
  mutate(PM2.5_violation = PM2.5 > 35, # WHO daily limit
         PM10_violation = PM10 > 50) %>%
  group_by(City) %>%
  summarise(PM2.5_violation_rate = mean(PM2.5_violation, na.rm = TRUE),
            PM10_violation_rate = mean(PM10_violation, na.rm = TRUE))

df %>%
  mutate(AQI_reduction = ifelse(Windspeed < 3, AQI * 0.9, AQI)) %>% # 10% reduction on stagnant days
  group_by(City) %>%
  summarise(avg_reduction = mean(AQI - AQI_reduction))

library(shiny)
ui <- fluidPage(
  selectInput("city", "Select City", unique(df$City)),
  plotOutput("aqi_plot")
)
server <- function(input, output) {
  output$aqi_plot <- renderPlot({
    df %>% filter(City == input$city) %>% 
      ggplot(aes(`Date Time`, AQI)) + geom_line()
  })
}
shinyApp(ui, server)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘libpng-dev’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘terra’, ‘raster’


Warning message in install.packages("leaflet"):
“installation of package ‘terra’ had non-zero exit status”
Warning message in install.packages("leaflet"):
“installation of package ‘raster’ had non-zero exit status”
Warning message in install.packages("leaflet"):
“installation of package ‘leaflet’ had non-zero exit status”

ERROR: Error in library(leaflet): there is no package called ‘leaflet’


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=360ce8b9-ca56-4969-a7c6-2b4d3b309009' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>